In [15]:
import pandas as pd
import numpy as np

In [16]:
stock_prices = pd.read_csv('files/train_files/stock_prices.csv', index_col=0, parse_dates=[1])
stock_list = pd.read_csv('files/stock_list.csv', parse_dates=[1])
#stock_list_incl = stock_list.where(stock_list.Universe0 == True]
#stock_list_incl.TradeDate = pd.to_datetime(stock_list_incl.TradeDate, format='%Y%m%d')

In [17]:
print(stock_prices['SupervisionFlag'].value_counts())
print(stock_prices['SupervisionFlag'].min(), stock_prices['SupervisionFlag'].max(), 
     stock_prices['SupervisionFlag'].shape)

False    2331036
True        1495
Name: SupervisionFlag, dtype: int64
False True (2332531,)


In [18]:
#Доля объемов на акцию от общедневного объема торговли
stock_prices = stock_prices.merge(stock_list[['SecuritiesCode', 'IssuedShares']], 
                                  how='left', on='SecuritiesCode')
stock_prices['Volume_share'] = stock_prices['Volume'] / stock_prices['IssuedShares']

In [19]:
#Log prices
stock_prices['Log_Close'] = np.log(stock_prices['Close'])
stock_prices['Log_Open'] = np.log(stock_prices['Open'])
stock_prices['Log_High'] = np.log(stock_prices['High'])
stock_prices['Log_Low'] = np.log(stock_prices['Low'])

In [48]:
stock_prices.sample(10)

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,IssuedShares,Volume_share,Log_Close,Log_Open,Log_High,Log_Low
212990,2017-06-19,8276,2407.0,2429.0,2402.0,2426.0,63600,1.0,NaN,False,-0.010134,58546470.0,0.001086,7.793999,7.786136,7.795235,7.784057
936662,2019-01-08,7840,932.0,932.0,914.0,918.0,33800,1.0,NaN,False,-0.001091,41397500.0,0.000816,6.822197,6.837333,6.837333,6.817831
1679785,2020-08-05,1928,2010.0,2015.5,1985.5,2007.5,1183100,1.0,NaN,False,-0.000749,684683466.0,0.001728,7.604645,7.605890,7.608623,7.593626
2126256,2021-07-02,8766,5136.0,5170.0,5127.0,5166.0,1110200,1.0,NaN,False,-0.001950,697500000.0,0.001592,8.549854,8.544030,8.550628,8.542276
814810,2018-10-02,4611,1329.0,1354.0,1315.0,1318.0,65600,1.0,NaN,False,0.003799,29710678.0,0.002208,7.183871,7.192182,7.210818,7.181592
1048776,2019-04-03,8012,1669.0,1678.0,1657.0,1677.0,182700,1.0,NaN,False,0.016148,124408285.0,0.001469,7.424762,7.419980,7.425358,7.412764
864592,2018-11-08,3983,5390.0,5480.0,5310.0,5430.0,64300,1.0,NaN,False,-0.189036,16606453.0,0.003872,8.599694,8.592301,8.608860,8.577347
688548,2018-06-27,4544,3215.0,3235.0,3180.0,3225.0,233800,1.0,NaN,False,0.029641,57446357.0,0.004070,8.078688,8.075583,8.081784,8.064636
485168,2018-01-22,4326,1416.0,1425.0,1404.0,1425.0,37400,1.0,NaN,False,0.000702,40426000.0,0.000925,7.261927,7.255591,7.261927,7.247081
666096,2018-06-11,6480,929.0,947.0,900.0,944.0,836600,1.0,NaN,False,-0.026178,73501425.0,0.011382,6.850126,6.834109,6.853299,6.802395


In [35]:
#which date has the most nas
na_close = stock_prices[stock_prices['Log_Close'].isna()]
grouped_na_close = na_close.groupby(
    ['Date']).agg({'SecuritiesCode' : 'count'}).reset_index()
print(grouped_na_close['SecuritiesCode'].value_counts())
print(grouped_na_close[grouped_na_close['SecuritiesCode'] == grouped_na_close['SecuritiesCode'].max()])

4       198
5       196
3       184
6       139
2       135
7        90
1        69
8        69
9        45
10       25
12       10
11       10
14        2
15        1
1988      1
13        1
Name: SecuritiesCode, dtype: int64
          Date  SecuritiesCode
889 2020-10-01            1988


In [47]:
#any outliers by volume
print(stock_prices['Volume_share'].describe())
print(stock_prices['Volume_share'].max())
big_volume = stock_prices[stock_prices['Volume_share'] > 1]
big_volume_by_date = big_volume.groupby(['Date']).agg({'SecuritiesCode' : 'count'}).reset_index()
big_volume_by_stock = big_volume.groupby(['SecuritiesCode']).agg({'Date' : 'count'}).reset_index()
print(big_volume_by_date['SecuritiesCode'].value_counts())
print(big_volume_by_stock['Date'].value_counts())

count    2.332531e+06
mean     5.352463e-03
std      2.200611e-02
min      0.000000e+00
25%      1.112540e-03
50%      2.343892e-03
75%      4.692912e-03
max      1.109317e+01
Name: Volume_share, dtype: float64
11.093171825676121
1    77
2    12
3     1
Name: SecuritiesCode, dtype: int64
1     12
2      4
9      3
3      2
6      2
5      2
11     1
8      1
10     1
Name: Date, dtype: int64


In [52]:
#how do the 0 volumes look
print(stock_prices[stock_prices['Volume'] == 0].shape)
stock_prices[stock_prices['Volume'] == 0].sample(20)

(7608, 17)


,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,IssuedShares,Volume_share,Log_Close,Log_Open,Log_High,Log_Low
1664135,2020-07-22,2761,NaN,NaN,NaN,NaN,0,1.0,NaN,False,-0.059122,11405400.0,0.0,NaN,NaN,NaN,NaN
1104114,2019-05-22,4365,NaN,NaN,NaN,NaN,0,1.0,NaN,False,0.000000,4512651.0,0.0,NaN,NaN,NaN,NaN
899544,2018-12-05,5945,NaN,NaN,NaN,NaN,0,1.0,NaN,False,-0.026114,5573817.0,0.0,NaN,NaN,NaN,NaN
1189524,2019-07-24,4365,NaN,NaN,NaN,NaN,0,1.0,NaN,False,0.008945,4512651.0,0.0,NaN,NaN,NaN,NaN
1986462,2021-03-22,9733,NaN,NaN,NaN,NaN,0,1.0,NaN,False,0.000000,10148409.0,0.0,NaN,NaN,NaN,NaN
2135460,2021-07-09,5945,NaN,NaN,NaN,NaN,0,1.0,NaN,False,0.000000,5573817.0,0.0,NaN,NaN,NaN,NaN
1423381,2020-01-23,1981,NaN,NaN,NaN,NaN,0,1.0,NaN,False,0.001267,11800000.0,0.0,NaN,NaN,NaN,NaN
1756225,2020-10-01,6788,NaN,NaN,NaN,NaN,0,1.0,NaN,False,0.037344,8656780.0,0.0,NaN,NaN,NaN,NaN
5996,2017-01-10,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,False,NaN,10000000.0,0.0,NaN,NaN,NaN,NaN
1980392,2021-03-17,9441,NaN,NaN,NaN,NaN,0,1.0,NaN,False,-0.006293,6732600.0,0.0,NaN,NaN,NaN,NaN
